In [2]:
import numpy as np
import pandas as pd

from openpyxl import load_workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from datetime import datetime
from pathlib import Path
from typing import Optional, List

In [3]:
arquivo = 'arquivo_base.xlsx'
df = pd.read_excel(arquivo)

In [4]:
def remover_colunas(df: pd.DataFrame, colunas_para_remover: list) -> pd.DataFrame:
    colunas_existentes = [col for col in colunas_para_remover if col in df.columns]
    return df.drop(columns=colunas_existentes)

In [5]:
def obter_df_shipping_line(df: pd.DataFrame) -> pd.DataFrame:
    df_shipping_line = (
        pd.DataFrame(df['SHIPPING LINE'].dropna().unique(), columns=['SHIPPING LINE'])
        .reset_index(drop=True)
    )
    return df_shipping_line

In [ ]:
def preparar_df_invoice(df: pd.DataFrame) -> pd.DataFrame:
    # Garante que os campos numéricos sejam numéricos de verdade
    df['BOXES'] = pd.to_numeric(df['BOXES'], errors='coerce')
    df['PALLET'] = pd.to_numeric(df['PALLET'], errors='coerce')
    df['PRICE'] = pd.to_numeric(df['PRICE'], errors='coerce')

    # Agrupa pela condição de variedade + preço unitário
    df_invoice = (
        df.groupby(['variety_invoice', 'PRICE'], as_index=False)
          .agg({
              'BOXES': 'sum',
              'PALLET': 'sum'
          })
          .rename(columns={
              'BOXES': 'Total',
              'PALLET': 'Total Pallets',
              'PRICE': 'Unit Price'
          })
    )

    # Arredonda pallets pra cima
    df_invoice['Total Pallets'] = np.ceil(df_invoice['Total Pallets']).astype(int)

    # Calcula valor total
    df_invoice['Total Amount'] = df_invoice['Total'] * df_invoice['Unit Price']

    # Reordena colunas
    df_invoice = df_invoice[[ 
        'variety_invoice', 'Unit Price', 'Total', 'Total Pallets', 'Total Amount'
    ]]

    # Arredonda valores para exibição
    df_invoice['Unit Price'] = df_invoice['Unit Price'].round(2)
    df_invoice['Total Amount'] = df_invoice['Total Amount'].round(2)

    return df_invoice

In [7]:
def obter_campos_distintos(df: pd.DataFrame):
    vessel = ', '.join(df['VESSEL'].dropna().unique())
    customer = ', '.join(df['CUSTOMER'].dropna().unique())
    port_dest = ', '.join(df['PORT OF DESTINATION'].dropna().unique())
    port_dep = ', '.join(df['PORT OF DEPARTURE'].dropna().unique())
    invoice_num = ', '.join(df['INVOICE NUMBER'].dropna().unique())

    return vessel, customer, port_dest, port_dep, invoice_num

In [8]:
def obter_df_containers(df: pd.DataFrame) -> pd.DataFrame:
    df_containers = (
        pd.DataFrame(df['CONTAINER'].dropna().unique(), columns=['CONTAINER'])
        .reset_index(drop=True)
    )
    return df_containers

### Funções

In [9]:
colunas = ['WEEK', 'DATE ARRIVAL', 'PLT SENSOR NUMBER', 'SEED', 'FINAL CLIENT',	
           'REFERENCE', 'TOTAL AMOUNT', 'CURRENCY', 'INCOTERM', 'FARM', 'TRACEABILITY', 
           'CLASS', 'PACKING', 'BRAND']

df_limpo = remover_colunas(df, colunas)

In [10]:
def obter_pesos_totais(df: pd.DataFrame):
    gross_weight_total = df['GROSS WEIGHT'].sum(skipna=True)
    net_weight_total = df['NET WEIGHT'].sum(skipna=True)
    return gross_weight_total, net_weight_total

In [11]:
df_limpo['variety_invoice'] = (
    df_limpo['VARIETY'].astype(str) +
    ' Melons In Natura ' +
    df_limpo['WEIGHT'].astype(str) + 'kg - x' +
    df_limpo['SIZE'].astype(str)
)

In [12]:
df_pronto = preparar_df_invoice(df_limpo)

vessel_str, customer_str, port_dest_str, port_dep_str, invoice_num_str = obter_campos_distintos(df_limpo)

df_containers = obter_df_containers(df_limpo)
df_shipping_line = obter_df_shipping_line(df_limpo)

### Dump no Excel

In [ ]:
def dump_containers_e_campos_fixos(
    xlsx_path: str,
    sheet_name: str,
    containers: Optional[List[str]] = None,
    invoice_num: str = "",
    vessel_str: str = "",
    port_dest_str: str = "",   # Shipping from
    port_dep_str: str = "",    # To
    df_invoice: Optional[pd.DataFrame] = None,
    invoice_cols_order: Optional[List[str]] = None,
    net_weight_total: Optional[float] = None,
    gross_weight_total: Optional[float] = None,
    start_cell: str = "H13",
    output_path: Optional[str] = None,
    push_down: bool = True
) -> str:
    wb = load_workbook(xlsx_path)
    ws = wb[sheet_name]

    # --- 1) Containers
    if containers is None:
        containers = []
    containers = [str(c).strip() for c in containers if c is not None and str(c).strip() != ""]
    n = len(containers)

    col_letters, start_row = coordinate_from_string(start_cell)
    start_col = column_index_from_string(col_letters)

    if push_down and n > 1:
        ws.insert_rows(idx=start_row + 1, amount=n - 1)

    for i, val in enumerate(containers):
        ws.cell(row=start_row + i, column=start_col, value=val)

    last_container_row = start_row + n - 1 if n > 0 else start_row - 1

    # --- 2) Invoice e data
    ws["E8"] = invoice_num
    hoje = datetime.today().strftime("%d/%m/%Y")
    ws["C10"] = f"Mossoró/RN, {hoje}"

    # --- 3) Vessel / Shipping from / To
    vessel_row = last_container_row + 2
    shipping_from_row = vessel_row + 2
    ws.cell(row=vessel_row, column=3, value=vessel_str)            # C
    ws.cell(row=shipping_from_row, column=3, value=port_dest_str)  # C
    ws.cell(row=shipping_from_row, column=8, value=port_dep_str)   # H (C + 5)

    # --- 4) Tabela df_invoice (B + E:F:G:H) + linha de total
    total_row_written = None
    if df_invoice is not None and not df_invoice.empty:
        if invoice_cols_order is None:
            invoice_cols_order = [
                'variety_invoice',
                'Total',
                'Total Pallets',
                'Unit Price',
                'Total Amount'
            ]
        df_tab = df_invoice.loc[:, invoice_cols_order].copy()

        # Linha de totais (AMOUNT CARTONS)
        total_row = {
            'variety_invoice': 'AMOUNT CARTONS',
            'Total': df_tab['Total'].sum(),
            'Total Pallets': df_tab['Total Pallets'].sum(),
            'Unit Price': df_tab['Unit Price'].mean(),
            'Total Amount': df_tab['Total Amount'].sum()
        }
        df_tab = pd.concat([df_tab, pd.DataFrame([total_row])], ignore_index=True)

        # Tipos numéricos garantidos
        for col in ['Total','Total Pallets','Unit Price','Total Amount']:
            df_tab[col] = pd.to_numeric(df_tab[col], errors='coerce')

        table_start_row = shipping_from_row + 7

        # Abre espaço para toda a tabela
        n_rows = len(df_tab)
        if push_down and n_rows > 1:
            ws.insert_rows(idx=table_start_row + 1, amount=n_rows - 1)

        # Mapas de coluna: B (descrição) e E..H (valores)
        col_B = 2  # B
        col_E = 5  # E
        col_F = 6  # F
        col_G = 7  # G
        col_H = 8  # H

        for i in range(n_rows):
            base_row = table_start_row + i
            # descrição
            desc_val = df_tab.iloc[i]['variety_invoice']
            ws.cell(row=base_row, column=col_B, value="" if pd.isna(desc_val) else str(desc_val))
            # valores
            ws.cell(row=base_row, column=col_E, value=None if pd.isna(df_tab.iloc[i]['Total']) else float(df_tab.iloc[i]['Total']))
            ws.cell(row=base_row, column=col_F, value=None if pd.isna(df_tab.iloc[i]['Total Pallets']) else float(df_tab.iloc[i]['Total Pallets']))
            ws.cell(row=base_row, column=col_G, value=None if pd.isna(df_tab.iloc[i]['Unit Price']) else float(df_tab.iloc[i]['Unit Price']))
            ws.cell(row=base_row, column=col_H, value=None if pd.isna(df_tab.iloc[i]['Total Amount']) else float(df_tab.iloc[i]['Total Amount']))

        # Guarda a linha onde o TOTAL (AMOUNT CARTONS) foi escrito
        total_row_written = table_start_row + n_rows - 1

     # --- 5) NET/GROSS WEIGHT (abaixo do AMOUNT CARTONS, na coluna C)
    if total_row_written is not None and (net_weight_total is not None or gross_weight_total is not None):
        # NÃO insere linhas; apenas escreve nos offsets pedidos
        net_row = total_row_written + 3 
        gross_row = net_row + 1

        if net_weight_total is not None:
            ws.cell(row=net_row, column=3, value=float(net_weight_total))      # Coluna C
        if gross_weight_total is not None:
            ws.cell(row=gross_row, column=3, value=float(gross_weight_total))  # Coluna C

    # --- Salva o arquivo
    if output_path is None:
        p = Path(xlsx_path)
        output_path = str(p.with_name(p.stem + "_out" + p.suffix))

    wb.save(output_path)
    return output_path

In [ ]:
containers_list = (
    df_containers['CONTAINER']
    .dropna()
    .astype(str)
    .str.strip()
    .tolist()
)

gross_total, net_total = obter_pesos_totais(df_limpo)

arquivo_final = dump_containers_e_campos_fixos(
    xlsx_path="arquivo_teste.xlsx",
    sheet_name="INVOICE_test",
    containers=containers_list,
    invoice_num=invoice_num_str,
    vessel_str=vessel_str,
    port_dest_str=port_dep_str,
    port_dep_str= port_dest_str,
    df_invoice=df_pronto,
    invoice_cols_order=['variety_invoice','Total','Total Pallets','Unit Price','Total Amount'],
    net_weight_total=net_total,
    gross_weight_total=gross_total,
    start_cell="H13",
    output_path="invoice_preenchida.xlsx",
    push_down=True
)

print("Gerado em:", arquivo_final)


Gerado em: invoice_preenchida.xlsx
